In [1]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

from soma import aims
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm



/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
a = ana.Anatomist()

existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jc225751'


create qapp
global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
loading module meshsplit
loading module palettecontrols
loading module volumepalettes
loading module save_resampled
loading module profilewindow
loading module statsplotwindow
loading module anacontrolmenu
loading module infowindow
loading module foldsplit
loading module simple_controls
loading module histogram
loading module valuesplotwindow
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


### Variable definitions

In [3]:
cluster_file = "/neurospin/dico/data/deep_folding/current/models/Champollion_V0/cluster/Sex_ukb_OLS_FColl-SRh_right_with-residualization.csv"

In [4]:
side = "R" # "R" or "L"
region = "F.Coll.-S.Rh." # "S.C.-sylv.", "ORBITAL", "CINGULATE", "SC-sylv", "F.I.P."
database='ukb'

In [5]:
cluster = pd.read_csv(f"{cluster_file}", index_col=0)

### Function definitions

In [6]:
def to_bucket(obj):
    if obj.type() == obj.BUCKET:
        return obj
    avol = a.toAimsObject(obj)
    c = aims.Converter(intype=avol, outtype=aims.BucketMap_VOID)
    abck = c(avol)
    bck = a.toAObject(abck)
    bck.releaseAppRef()
    return bck

def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()

def buckets_average(subject_id_list, dataset_name_list):
    dic_vol = {}
    dim = 0
    rep = 0
    if len(subject_id_list) == 0:
        return False
    while dim == 0 and rep < len(subject_id_list):
        if dataset_name_list[rep].lower() in ['ukb', 'ukbiobank', 'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset_name_list[rep].lower() in ['hcp']:
            dataset = 'hcp'
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"
        if os.path.isfile(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz'):
            sum_vol = aims.read(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz').astype(float)
            dim = sum_vol.shape
            sum_vol.fill(0)
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id_list[0]}_cropped_skeleton.nii.gz')
        rep += 1

    for subject_id, dataset in zip(subject_id_list,dataset_name_list):
        if dataset.lower() in ['ukb', 'ukbiobank',  'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset.lower() == 'hcp':
            dataset = 'hcp'
            
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"

        if os.path.isfile(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz'):
            vol = aims.read(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            # compare the dim with the first file dim

            if vol.np.shape != dim:
                raise ValueError(f"{subject_id_list[0]} and {subject_id} must have the same dim")

                
            # to have a binary 3D structure
            dic_vol[subject_id] = (vol.np > 0).astype(int)
            sum_vol.np[:] += (vol.np > 0).astype(int) 
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')

    sum_vol = sum_vol / len(subject_id_list)
    print(sum_vol, sum_vol.shape)
    return sum_vol

In [7]:
def visualize_averages_along_cluster(df, column_name, database, nb_subjects_per_average=200, nb_columns=5, nb_lines=1):
    # anatomist objects
    global _block
    global _average_dic
    global dic_packages
    _average_dic = {}


    # Order according to cluster
    # Removes cluster 0, which is the non-cluster
    df = df[df[column_name]!=0]
    df = df.sort_values(column_name)
    
    # Computes umber of columns and number oif subjects per average
    min_size = df.groupby(column_name).size().min()
    nb_subjects_per_average = min(min_size, nb_subjects_per_average)
    nb_columns = df[column_name].unique().shape[0]
    print(f"nb subjects per average = {nb_subjects_per_average}")
    
    # Takes the first nb_subjects_per_average elements of each cluster
    df = df.groupby(column_name, as_index=False).head(nb_subjects_per_average)
    
    cluster_number = df[[column_name]]
    
    # Creates block window with number of columns = nb of clusters
    _block = a.createWindowsBlock(nb_columns)
    nb_windows = nb_columns * nb_lines

    # Creates dictionary of subjects to average
    dic_packages = {}
    for i in np.unique(cluster_number):
        list_idx = cluster_number[cluster_number[column_name]==i].index.to_numpy()
        dic_packages[i] = list_idx
    
    list_database = [database for i in range(nb_subjects_per_average)]
    n_pack = len(dic_packages)

    # Loop of display averages
    list_pack = np.unique(cluster_number)
    for i in list_pack:
        sum_vol = buckets_average(dic_packages[i], list_database)
        _average_dic[f'a_sum_vol{i}'] = a.toAObject(sum_vol)
        _average_dic[f'a_sum_vol{i}'].setPalette(minVal=0, absoluteMode=True)
        #wsum = a.createWindow('Sagittal', block=block)
        #wsum.addObjects(a_sum_vol)
        _average_dic[f'rvol{i}'] = a.fusionObjects(
            objects=[_average_dic[f'a_sum_vol{i}']],
            method='VolumeRenderingFusionMethod')
        _average_dic[f'rvol{i}'].releaseAppRef()
        # custom palette
        n = len(dic_packages[i])
        pal = a.createPalette('VR-palette')
        pal.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.5/n};0;1;1'
        build_gradient(pal)
        _average_dic[f'rvol{i}'].setPalette('VR-palette', minVal=0.05, absoluteMode=True)
        pal2 = a.createPalette('slice-palette')
        pal2.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.3/n};0;{0.7/n};1;1;1'
        build_gradient(pal2)
        _average_dic[f'a_sum_vol{i}'].setPalette('slice-palette')
        # rvol.palette().fill()
        _average_dic[f'wvr{i}'] = a.createWindow('3D', block=_block)
        _average_dic[f'wvr{i}'].addObjects(_average_dic[f'rvol{i}'])

### Visualization

In [8]:
# block = a.createWindowsBlock(10)
visualize_averages_along_cluster(cluster, "cluster", database, nb_subjects_per_average=200)

nb subjects per average = 200
<soma.aims.Volume_DOUBLE object at 0x756b74bb81f0> (34, 62, 41, 1)
Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 4
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader foun

# Displaying subjects

In [9]:
# list_subjects = list_smallest
# block = a.createWindowsBlock(5) # 5 columns
# dic_windows = {}
# for subject in list_subjects:
#     path_to_t1mri = f'/neurospin/dico/data/bv_databases/human/not_labeled/hcp/hcp/{subject}/t1mri/BL'
#     dic_windows[f'w{subject}'] = a.createWindow("3D", block=block)
#     dic_windows[f'white_{subject}'] = a.loadObject(f'{path_to_t1mri}/default_analysis/segmentation/mesh/{subject}_{side}white.gii')
#     dic_windows[f'white_{subject}'].loadReferentialFromHeader()
#     dic_windows[f'sulci_{subject}'] = a.loadObject(f'{path_to_t1mri}/default_analysis/folds/3.1/{side}{subject}.arg')
#     dic_windows[f'sulci_{subject}'].loadReferentialFromHeader()
#     dic_windows[f'w{subject}'].addObjects([dic_windows[f'white_{subject}'], dic_windows[f'sulci_{subject}']])

In [10]:
# df.sort_values("predicted")[::10]

In [11]:
# df_concat = pd.concat([df.sort_values("predicted")[::10][:5], df.sort_values("predicted")[::10][-5:]])

In [12]:
# df_concat

In [13]:
# visualize_averages_along_parameter(df_concat, "predicted", database, nb_subjects_per_average=1, nb_columns=5, nb_lines=2)

no position could be read at 302, 258
